Utilizei Python 3 e a biblioteca pandas.  Os códigos rodam no Jupyter Notebook. 

In [1]:
import pandas as pd

Aqui importo a biblioteca pandas e a nomeamos como 'pd' para facilitar a digitação do código.

In [2]:
xls = pd.ExcelFile('SIC 584922017103.xlsx')

Como é um arquivo Excel, uso a classe ExcelFile e chamo o dataframe de "xls".

In [3]:
xls.sheet_names

['Base de Dados', 'Metodologia', 'Pedido', 'Campos da Tabela']

Aqui, verifico as tabelas que existem dentro do arquivo. Existem 4 e as que me interessaram foi "Base de Dados" e "Campos da Tabela".

In [4]:
xls.parse('Base de Dados')

,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,NOME_DELEGACIA,CIDADE,ANO_BO,NUM_BO,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,...,DESCRICAO_DEFICIENCIA,DESCR_RELACIONAMENTO,FLAG_VITIMA_FATAL,SEXO_PESSOA,IDADE_PESSOA,COR_CUTIS,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,NACIONALIDADE_PESSOA,NATURALIDADE_PESSOA
0,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,43.0,Parda,PRENDAS DOMESTICAS,1 Grau completo,BRASILEIRA,ORLANDIA -SP
1,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,19.0,Parda,ESTUDANTE,1 Grau completo,BRASILEIRA,NOVA GRANADA -SP
2,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,43.0,Parda,PRENDAS DOMESTICAS,1 Grau completo,BRASILEIRA,ORLANDIA -SP
3,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,4,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,19.0,Parda,ESTUDANTE,1 Grau completo,BRASILEIRA,NOVA GRANADA -SP
4,100217,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,ALTAIR,2019,12,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BARRETOS,DEL.POL.ALTAIR,...,NaN,NaN,NaN,F,43.0,Branca,PRENDAS DOMESTICAS,1 Grau incompleto,BRASILEIRA,NOSSA SENHORA GLORIA -SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57773,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,391,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,UNIAO ESTAVEL,N,F,27.0,Branca,LAVRADOR(A),1 Grau completo,BRASILEIRA,MORRO DO CHAPÉU/BA
57774,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,391,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,NENHUMA RELACAO,N,F,28.0,Parda,LAVRADOR(A),1 Grau incompleto,BRASILEIRA,BARRINHA -SP
57775,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,391,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,UNIAO ESTAVEL,N,F,27.0,Branca,LAVRADOR(A),1 Grau completo,BRASILEIRA,MORRO DO CHAPÉU/BA
57776,100912,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.PLANTÃO BEBEDOURO,VISTA ALEGRE DO ALTO,2019,620,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.BEBEDOURO,DEL.POL.VISTA ALEGRE DO ALTO,...,NaN,NaN,NaN,F,25.0,Branca,FISIOTERAPEUTA,Superior completo,BRASILEIRA,PIRANGI -SP


Comando para checar o que compõe a tabela "Base de Dados". É realmente o que eu preciso, mas são muitas colunas (53), então é necessário uma filtragem.

In [5]:
xls.parse('Campos da Tabela')

,A PRESENTE TABELA TEM POR FINALIDADE ESCLARECER OS CAMPOS CONTIDOS NA BASE DE DADOS,Unnamed: 1
0,*Resultado Null indica que o campo se encontra...,NaN
1,NaN,NaN
2,Campos,Descrição
3,NOME_DEPARTAMENTO,Departamento responsável pelo registro
4,NOME_SECCIONAL,Delegacia Seccional responsável pelo registro
5,NOME_DELEGACIA,Delegacia responsável pelo registro
6,CIDADE,Cidade de Registro
7,ANO_BO,Ano do BO
8,NUM_BO,Número do BO
9,NOME_DEPARTAMENTO_CIRC,Departamento de Circunscrição


A tabela "Campos da Tabela" nos mostra o que significa cada coluna, então ficará mais fácil pra gente entender cada uma e filtrar o que é realmente necessário para a história que quero contar.

In [6]:
campos = pd.read_excel(xls, 'Campos da Tabela')

Criei um novo dataframe chamado "campos" somente com a tabela "Campos da Tabela", pois há muito espaço e dificultou a leitura de algumas linhas, então irei limpar.

In [7]:
campos.columns

Index(['A PRESENTE TABELA TEM POR FINALIDADE ESCLARECER OS CAMPOS CONTIDOS NA BASE DE DADOS', 'Unnamed: 1'], dtype='object')

Aqui, confirmei o nome das colunas. Como não foi especificado na tabela, foi usado a primeira linha como nome de cada coluna. 

In [8]:
campos = campos.rename(columns={'A PRESENTE TABELA TEM POR FINALIDADE ESCLARECER OS CAMPOS CONTIDOS NA BASE DE DADOS':'CAMPO'})
campos = campos.rename(columns={'Unnamed: 1':'DESCRIÇÃO'})
campos = campos.drop([0, 1, 2])

In [9]:
campos

,CAMPO,DESCRIÇÃO
3,NOME_DEPARTAMENTO,Departamento responsável pelo registro
4,NOME_SECCIONAL,Delegacia Seccional responsável pelo registro
5,NOME_DELEGACIA,Delegacia responsável pelo registro
6,CIDADE,Cidade de Registro
7,ANO_BO,Ano do BO
8,NUM_BO,Número do BO
9,NOME_DEPARTAMENTO_CIRC,Departamento de Circunscrição
10,NOME_SECCIONAL_CIRC,Seccional de Circunscrição
11,NOME_DELEGACIA_CIRC,Delegacia de Circunscrição
12,NOME_MUNICIPIO_CIRC,Município da Delegacia de Circunscrição


 Alterei o nome das colunas para algo mais amigável (CAMPO E DESCRIÇÃO) e retirei as três primeiras linhas da tabela.

In [10]:
df = pd.read_excel(xls, 'Base de Dados')

Criei um novo dataframe chamado "df" somente com a tabela "Base de Dados".

In [11]:
df.sample(5)

,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,NOME_DELEGACIA,CIDADE,ANO_BO,NUM_BO,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,...,DESCRICAO_DEFICIENCIA,DESCR_RELACIONAMENTO,FLAG_VITIMA_FATAL,SEXO_PESSOA,IDADE_PESSOA,COR_CUTIS,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO,NACIONALIDADE_PESSOA,NATURALIDADE_PESSOA
50148,900020,DIPOL - DEPTO DE INTELIGENCIA,DELEGACIA ELETRONICA,DELEGACIA ELETRONICA,S.PAULO,2020,757941,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.SAO CARLOS,02º D.P. SÃO CARLOS,...,NaN,NaN,NaN,F,33.0,NaN,ESTETICISTA,NaN,NaN,BRASILEIRA
4997,100908,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.ARARAQUARA,DEL.SEC.ARARAQUARA PLANTÃO,ARARAQUARA,2019,1167,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.ARARAQUARA,02º D.P. ARARAQUARA,...,NaN,NaN,NaN,F,59.0,Branca,DESENHISTA,NaN,BRASILEIRA,ARARAQUARA -SP
15207,100408,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.SERTÃOZINHO,DEL.POL.DUMONT,DUMONT,2019,344,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.SERTÃOZINHO,DEL.POL.DUMONT,...,NaN,NaN,NaN,F,19.0,Parda,ESTUDANTE,NaN,BRASILEIRA,GUARIBA -SP
52710,100431,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.RIBEIRAO PRETO,DDM RIBEIRAO PRETO,SALES OLIVEIRA,2019,1921,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.S. JOAQUIM DA BARRA,DEL.POL.SALES OLIVEIRA,...,NaN,CONHECIDO,N,F,64.0,Branca,PRENDAS DOMESTICAS,NaN,BRASILEIRA,CRAVINHOS -SP
15169,100408,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.SERTÃOZINHO,DEL.POL.DUMONT,DUMONT,2019,141,DEINTER 3 - RIBEIRAO PRETO,DEL.SEC.SERTÃOZINHO,DEL.POL.DUMONT,...,NaN,UNIAO ESTAVEL,N,F,35.0,Parda,COZINHEIRO(A),2 Grau completo,BRASILEIRA,RIBEIRAO PRETO -SP


Mesmo com a descrição de cada campo, algumas coisas eu ainda não consegui entender, então peguei 5 exemplos aleatórios para analisar o que cada um significa e selecionar melhor as colunas que desejo manter.

In [12]:
df = df[['CIDADE', 'DATA_COMUNICACAO_BO', 'FLAG_STATUS', 'VIOLENCIA_DOMESTICA', 'RUBRICA', 'DESCR_SUBTIPOLOCAL', 'DESCR_RELACIONAMENTO', 'IDADE_PESSOA', 'DESCR_PROFISSAO', 'DESCR_GRAU_INSTRUCAO']]

In [13]:
df.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
47927,S.CARLOS,2019-05-10,Consumado,NaN,Injúria (art. 140),CASA,NaN,20.0,DESEMPREGADO(A),NaN
49837,S.CARLOS,2019-11-22,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,44.0,EDUCADOR,Superior completo
1769,ARARAQUARA,2019-02-04,Consumado,NaN,Lesão corporal (art. 129),CASA,NaN,29.0,SECRETARIO(A),2 Grau completo
5347,ARARAQUARA,2019-10-13,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),CASA,PARENTESCO,38.0,NaN,NaN
27366,JABOTICABAL,2019-04-08,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,PARENTESCO,14.0,ESTUDANTE,1 Grau incompleto


Agora apenas com as colunas que desejo, fica mais fácil a análise dos dados. A forma como a idade está apresentada me incomoda um pouco, então vou corrigir.

In [14]:
df.dtypes

CIDADE                          object
DATA_COMUNICACAO_BO     datetime64[ns]
FLAG_STATUS                     object
VIOLENCIA_DOMESTICA             object
RUBRICA                         object
DESCR_SUBTIPOLOCAL              object
DESCR_RELACIONAMENTO            object
IDADE_PESSOA                   float64
DESCR_PROFISSAO                 object
DESCR_GRAU_INSTRUCAO            object
dtype: object

O tipo da coluna está como float, mas vou mudar para inteiro, já que ninguem diz "minha idade é 51,3".

In [15]:
import numpy as np

In [16]:
df['IDADE_PESSOA'] = df['IDADE_PESSOA'].fillna(0).astype(np.int64)

Importei a biblioteca numpy como "np" e fiz a mudança do tipo da coluna, colocando "0" nas que não apresentavam valores.

In [17]:
df.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
55790,TAIUVA,2019-12-09,Consumado,NaN,Difamação (art. 139),WHATSAPP,NaN,15,ESTUDANTE,2 Grau incompleto
50060,S.CARLOS,2020-04-06,Consumado,NaN,Ameaça (art. 147),CASA,NaN,23,ESTUDANTE,NaN
50380,S.JOAQUIM DA BARRA,2019-07-05,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,46,PRENDAS DOMESTICAS,1 Grau incompleto
23600,IBITINGA,2019-02-20,Consumado,NaN,Ameaça (art. 147),OUTROS,NaN,35,EMPRESARIO(A),NaN
10537,BEBEDOURO,2020-03-05,Consumado,NaN,Calúnia (art. 138),APLICATIVOS DE MENSAGENS,NaN,27,PRENDAS DOMESTICAS,2 Grau completo


In [18]:
df.to_csv('violencia-domestica-rp.csv', encoding='utf8', index=False)

É isso mesmo que quero, então aproveitei pra exportar a tabela caso eu queira utilizar novamente uma outra hora.

Como eu quero falar especificamente do período da quarentena, preciso separar as datas por mês. A coluna "DATA_COMUNICACAO_BO" já está com o tipo correto, como vi anteriormente com o comando dtype, logo vai ser simples.

In [19]:
df2019 = df[df['DATA_COMUNICACAO_BO'].dt.year == 2019]

In [20]:
df2019.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
46651,RIFAINA,2019-01-22,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,ENVOLVIMENTO AMOROSO,25,DESEMPREGADO(A),1 Grau completo
53609,SERTAOZINHO,2019-02-04,Consumado,NaN,Lesão corporal (art. 129),CASA,PARENTESCO,42,PROFESSOR(A),Superior completo
27838,JARDINOPOLIS,2019-05-30,Consumado,NaN,Injúria (art. 140),AREA COMUM-,NaN,68,PRENDAS DOMESTICAS,1 Grau incompleto
6597,BARRETOS,2019-08-29,Consumado,NaN,Ameaça (art. 147),CASA,NaN,61,PENSIONISTA,1 Grau incompleto
18911,FRANCA,2019-10-02,Consumado,NaN,Ameaça (art. 147),CASA,NaN,45,NaN,NaN


Primeiro, separei por ano. Irei fazer o mesmo com 2020. Perceba que criei um novo dataframe chamado "df2019" para não sobreescrever as informações do outro dataframe, criando também um novo dataframe, mas chamado "df2020"

In [21]:
df2020 = df[df['DATA_COMUNICACAO_BO'].dt.year == 2020]

In [22]:
df2020.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
32773,OLIMPIA,2020-05-12,Consumado,NaN,Dano (art. 163),CASA,NaN,35,NAO INFORMADA,NaN
41149,RIBEIRAO PRETO,2020-03-02,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,ENVOLVIMENTO AMOROSO,18,PRENDAS DOMESTICAS,NaN
49385,S.CARLOS,2020-06-23,Consumado,NaN,Injúria (art. 140),CASA,NaN,27,CONTADOR(A),Superior completo
55555,TABATINGA,2020-04-03,Consumado,NaN,Lesão corporal (art. 129),VIA PÚBLICA,NaN,34,COSTUREIRO(A),2 Grau completo
17922,FRANCA,2020-06-19,Consumado,NaN,Ameaça (art. 147),CASA,NaN,19,ESTUDANTE,NaN


In [23]:
df2019 = df2019.sort_values(by='DATA_COMUNICACAO_BO')
df2020 = df2020.sort_values(by='DATA_COMUNICACAO_BO')

In [24]:
df2019.tail(3)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
5526,ARARAQUARA,2019-12-31,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,PARENTESCO,56,PRENDAS DOMESTICAS,2 Grau completo
8107,BARRETOS,2019-12-31,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),APARTAMENTO,ENVOLVIMENTO AMOROSO,36,LAVRADOR(A),1 Grau incompleto
5530,ARARAQUARA,2019-12-31,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,PARENTESCO,79,APOSENTADO(A),Analfabeto


In [25]:
df2020.tail(3)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
41805,RIBEIRAO PRETO,2020-06-30,Consumado,NaN,Injúria (art. 140),APARTAMENTO,NaN,67,NaN,NaN
27304,JABOTICABAL,2020-06-30,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),AREA COMUM-,UNIAO ESTAVEL,33,PRENDAS DOMESTICAS,2 Grau completo
27305,JABOTICABAL,2020-06-30,Consumado,NaN,Estupro de vulneravel (art.217-A),CASA,NaN,7,NaN,NaN


Primeiro, eu ordenei por data, do mês 1 ao 12. Depois, eu vi o final de cada uma para ver até qual mês iria. Como vai até o mês de julho na na tabela de 2020, farei o mesmo filtro na de 2019.

In [26]:
dfquar19 = df2019[(df2019['DATA_COMUNICACAO_BO'] >= '2019-03-01') & (df2019['DATA_COMUNICACAO_BO'] <= '2019-06-30')]

In [27]:
dfquar20 = df2020[(df2020['DATA_COMUNICACAO_BO'] >= '2020-03-01') & (df2020['DATA_COMUNICACAO_BO'] <= '2020-06-30')]

In [28]:
dfquar19.head(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
28677,MATAO,2019-03-01,Consumado,NaN,Dano (art. 163),CASA,NaN,59,NAO INFORMADA,Superior completo
1286,ARARAQUARA,2019-03-01,Consumado,NaN,Ameaça (art. 147),CASA,NaN,75,COSTUREIRO(A),1 Grau incompleto
52303,S.SIMAO,2019-03-01,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),VIA PÚBLICA,ENVOLVIMENTO AMOROSO,31,NaN,NaN
33170,ORLANDIA,2019-03-01,Consumado,NaN,Lesão corporal (art. 129),POSTO DE GASOLINA,NaN,24,DESEMPREGADO(A),2 Grau incompleto
1285,ARARAQUARA,2019-03-01,Consumado,NaN,Injúria (art. 140),PREFEITURA MUNICIPAL,NaN,29,FUNCION.PUBLICO MUNICIPAL,Superior completo


In [29]:
dfquar20.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
21914,GUATAPARA,2020-06-18,Consumado,NaN,Estupro de vulneravel (art.217-A),CASA,NaN,9,ESTUDANTE,NaN
52524,S.SIMAO,2020-03-09,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),LOJA DE BRINQUEDOS,CASAMENTO,44,NaN,NaN
23367,IBITINGA,2020-03-12,Consumado,NaN,A.I.-Lesão corporal (art. 129),VIA PÚBLICA,NaN,16,ESTUDANTE,1 Grau completo
23435,IBITINGA,2020-05-04,Consumado,NaN,A.I.-Lesão corporal (art. 129),CASA,NaN,62,APOSENTADO(A),Analfabeto
41601,RIBEIRAO PRETO,2020-05-15,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),APARTAMENTO,UNIAO ESTAVEL,30,BALCONISTA,NaN


Agora que tenho somente os meses que preciso, posso começar com minha matéria, fazendo a leitura dos dados nos dataframes específicos.

In [30]:
dfquar19.to_csv('quar19.csv', index=False, encoding ='utf8')
dfquar20.to_csv('quar20.csv', index=False, encoding ='utf8')

In [31]:
dfquar19.shape

(13774, 10)

In [32]:
dfquar20.shape

(9876, 10)

Usei o comando shape para ver a quantidade de linhas em cada tabela, estimado, assim, o total de casos de cada ano.

In [33]:
dfquar19['CIDADE'].unique()

array(['MATAO                                   ',
       'ARARAQUARA                              ',
       'S.SIMAO                                 ',
       'ORLANDIA                                ',
       'LUIS ANTONIO                            ',
       'MIGUELOPOLIS                            ',
       'PORTO FERREIRA                          ',
       'RIFAINA                                 ',
       'FRANCA                                  ',
       'CANDIDO RODRIGUES                       ',
       'GUAIRA                                  ',
       'CRAVINHOS                               ',
       'BATATAIS                                ',
       'BARRETOS                                ',
       'JARDINOPOLIS                            ',
       'COLOMBIA                                ',
       'S.ROSA DE VITERBO                       ',
       'MORRO AGUDO                             ',
       'IBITINGA                                ',
       'ITUVERAVA              

Preciso filtrar por cidade, então tenho que ter certeza que os nomes estão escritos corretamente. Há um grande espaço neles, então irei remover.

In [34]:
dfquar19['CIDADE'] = dfquar19['CIDADE'].map(str.strip)

<ipython-input-34-d0050541a4c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfquar19['CIDADE'] = dfquar19['CIDADE'].map(str.strip)


In [35]:
dfquar19['CIDADE'].unique()

array(['MATAO', 'ARARAQUARA', 'S.SIMAO', 'ORLANDIA', 'LUIS ANTONIO',
       'MIGUELOPOLIS', 'PORTO FERREIRA', 'RIFAINA', 'FRANCA',
       'CANDIDO RODRIGUES', 'GUAIRA', 'CRAVINHOS', 'BATATAIS', 'BARRETOS',
       'JARDINOPOLIS', 'COLOMBIA', 'S.ROSA DE VITERBO', 'MORRO AGUDO',
       'IBITINGA', 'ITUVERAVA', 'GUARA', 'BORBOREMA', 'OLIMPIA', 'ALTAIR',
       'PATROCINIO PAULISTA', 'RIBEIRAO PRETO', 'JABOTICABAL', 'S.CARLOS',
       'SERTAOZINHO', 'BARRINHA', 'ITAPOLIS', 'BEBEDOURO',
       'FERNANDO PRESTES', 'PONTAL', 'PEDREGULHO', 'BRODOWSKI', 'TAIACU',
       'IGARAPAVA', 'DOURADO', 'S.JOAQUIM DA BARRA', 'SERRANA',
       'MONTE ALTO', 'AMERICO BRASILIENSE', 'TAQUARITINGA', 'GUATAPARA',
       'PITANGUEIRAS', 'VIRADOURO', 'PIRANGI', 'MONTE AZUL PAULISTA',
       'TAIUVA', 'ALTINOPOLIS', 'S.LUCIA', 'DESCALVADO', 'DUMONT',
       'S.RITA PASSA QUATRO', 'CAJURU', 'GUARACI', 'GUARIBA',
       'SERRA AZUL', 'IBATE', 'IPUA', 'RIBEIRAO BONITO', 'NOVA EUROPA',
       'NUPORANGA', 'SALES OLIVE

Foi me dado um aviso, mas funcionou, então irei ignorar. Agora, filtrar.

In [36]:
dfquar19rp = dfquar19[dfquar19['CIDADE']=='RIBEIRAO PRETO']

In [37]:
dfquar19rp.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
38290,RIBEIRAO PRETO,2019-05-07,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,UNIAO ESTAVEL,25,DAMA DE COMPANHIA,2 Grau incompleto
43007,RIBEIRAO PRETO,2019-05-29,Consumado,NaN,Ameaça (art. 147),ENSINO FUNDAMENTAL,NaN,11,ESTUDANTE,NaN
38707,RIBEIRAO PRETO,2019-06-17,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),CASA,UNIAO ESTAVEL,28,DESEMPREGADO(A),1 Grau incompleto
42360,RIBEIRAO PRETO,2019-03-08,Consumado,NaN,Difamação (art. 139),CASA,NaN,52,ATENDENTE,2 Grau completo
36856,RIBEIRAO PRETO,2019-03-28,Consumado,NaN,A.I.-Ameaça (art. 147),CASA,NaN,50,PRENDAS DOMESTICAS,2 Grau completo


Agora tenho somente os crimes cometidos em Ribeirão Preto. Irei fazer mais algumas filtragens, com o mesmo comando usado anteriormente, e também no dataframe dfquar20. Achei melhor separar os dataframes, colocando um "rp" na frente.

In [38]:
dfquar19rp.shape

(2320, 10)

In [39]:
dfquar20['CIDADE'].unique()

array(['RIBEIRAO PRETO                          ',
       'MONTE AZUL PAULISTA                     ',
       'GUARIBA                                 ',
       'ITUVERAVA                               ',
       'BARRETOS                                ',
       'S.JOAQUIM DA BARRA                      ',
       'JARDINOPOLIS                            ',
       'SERTAOZINHO                             ',
       'CRAVINHOS                               ',
       'FRANCA                                  ',
       'IBITINGA                                ',
       'S.CARLOS                                ',
       'TAQUARITINGA                            ',
       'BATATAIS                                ',
       'ARARAQUARA                              ',
       'SERRANA                                 ',
       'PORTO FERREIRA                          ',
       'IGARAPAVA                               ',
       'RIBEIRAO BONITO                         ',
       'CAJOBI                 

In [40]:
dfquar20['CIDADE'] = dfquar20['CIDADE'].map(str.strip)

<ipython-input-40-f37ce8656960>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfquar20['CIDADE'] = dfquar20['CIDADE'].map(str.strip)


In [41]:
dfquar20['CIDADE'].unique()

array(['RIBEIRAO PRETO', 'MONTE AZUL PAULISTA', 'GUARIBA', 'ITUVERAVA',
       'BARRETOS', 'S.JOAQUIM DA BARRA', 'JARDINOPOLIS', 'SERTAOZINHO',
       'CRAVINHOS', 'FRANCA', 'IBITINGA', 'S.CARLOS', 'TAQUARITINGA',
       'BATATAIS', 'ARARAQUARA', 'SERRANA', 'PORTO FERREIRA', 'IGARAPAVA',
       'RIBEIRAO BONITO', 'CAJOBI', 'JABOTICABAL', 'BEBEDOURO',
       'DESCALVADO', 'BOA ESPERANCA DO SUL', 'ITAPOLIS', 'COLOMBIA',
       'S.SIMAO', 'S.RITA PASSA QUATRO', 'PIRANGI', 'COLINA',
       'PATROCINIO PAULISTA', 'CRISTAIS PAULISTA', 'RINCAO',
       'VISTA ALEGRE DO ALTO', 'S.ANTONIO DA ALEGRIA', 'CAMPINAS',
       'CAJURU', 'IBATE', 'NUPORANGA', 'SEVERINIA', 'IPUA',
       'PITANGUEIRAS', 'FERNANDO PRESTES', 'MONTE ALTO', 'MATAO',
       'GUAIRA', 'GUARA', 'SERRA AZUL', 'LUIS ANTONIO', 'BARRINHA',
       'AMERICO BRASILIENSE', 'MIGUELOPOLIS', 'TABATINGA', 'JERIQUARA',
       'GUARACI', 'JABORANDI', 'PONTAL', 'S.JOSE DA BELA VISTA',
       'GUATAPARA', 'OLIMPIA', 'S.ROSA DE VITERBO', 'MORR

In [42]:
dfquar20rp = dfquar20[dfquar20['CIDADE']=='RIBEIRAO PRETO']

In [43]:
dfquar20rp.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
45621,RIBEIRAO PRETO,2020-03-15,Consumado,NaN,Injúria (art. 140),CASA,NaN,55,CABELEIREIRO(A),2 Grau incompleto
28331,RIBEIRAO PRETO,2020-05-04,Consumado,NaN,Ameaça (art. 147),VIA PÚBLICA,NaN,26,AUXILIAR DE CARTORIO,NaN
46311,RIBEIRAO PRETO,2020-04-23,Consumado,NaN,Ameaça (art. 147),OUTROS,NaN,25,AUXILIAR DE COZINHA,NaN
46205,RIBEIRAO PRETO,2020-04-03,Consumado,NaN,Lesão corporal (art. 129),OUTROS,NaN,41,PROFESSOR(A),NaN
46407,RIBEIRAO PRETO,2020-05-20,Consumado,NaN,Difamação (art. 139),OUTROS,NaN,40,COORDENADOR(A),NaN


In [44]:
dfquar20rp.shape

(1551, 10)

In [45]:
dfquar19rp['VIOLENCIA_DOMESTICA'].unique()

array([nan, 'VIOLÊNCIA DOMESTICA'], dtype=object)

In [46]:
dfquar19rpvd = dfquar19rp[dfquar19rp['VIOLENCIA_DOMESTICA']=='VIOLÊNCIA DOMESTICA']

In [47]:
dfquar19rpvd.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
37908,RIBEIRAO PRETO,2019-04-02,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,46,AUXILIAR DE LIMPEZA,1 Grau completo
42473,RIBEIRAO PRETO,2019-03-23,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),CASA,ENVOLVIMENTO AMOROSO,20,BABÁ,1 Grau incompleto
38692,RIBEIRAO PRETO,2019-06-13,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,UNIAO ESTAVEL,26,ENFERMEIRO(A),Superior completo
38051,RIBEIRAO PRETO,2019-04-16,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,29,DAMA DE COMPANHIA,2 Grau completo
38701,RIBEIRAO PRETO,2019-06-17,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,ENVOLVIMENTO AMOROSO,20,SECRETARIO(A),2 Grau completo


In [48]:
dfquar19rpvd.shape

(1112, 10)

In [49]:
dfquar20rp['VIOLENCIA_DOMESTICA'].unique()

array([nan, 'VIOLÊNCIA DOMESTICA'], dtype=object)

In [50]:
dfquar20rpvd = dfquar20rp[dfquar20rp['VIOLENCIA_DOMESTICA']=='VIOLÊNCIA DOMESTICA']

In [51]:
dfquar20rpvd.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,DESCR_RELACIONAMENTO,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
41474,RIBEIRAO PRETO,2020-04-16,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,31,OPERADOR DE EMPILHADEIRA,Superior completo
41305,RIBEIRAO PRETO,2020-03-17,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art 129 § 9º),ALBERGUE-OUTROS,ENVOLVIMENTO AMOROSO,24,DESEMPREGADO(A),1 Grau completo
41769,RIBEIRAO PRETO,2020-06-23,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,35,AUXILIAR DE LIMPEZA,NaN
45890,RIBEIRAO PRETO,2020-06-12,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),VIA PÚBLICA,UNIAO ESTAVEL,23,MANICURE,NaN
41179,RIBEIRAO PRETO,2020-03-05,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,CASAMENTO,38,AUXILIAR DE LIMPEZA,NaN


In [52]:
dfquar20rpvd.shape

(818, 10)

In [53]:
dfquar19rpvd['DESCR_RELACIONAMENTO'].unique()

array(['CASAMENTO                               ',
       'CONHECIDO                               ',
       'UNIAO ESTAVEL                           ',
       'ENVOLVIMENTO AMOROSO                    ',
       'PARENTESCO                              ',
       'VIZINHANCA                              ',
       'NENHUMA RELACAO                         ',
       'IGNORADO                                '], dtype=object)

In [54]:
dfquar19rpvd['DESCR_RELACIONAMENTO'] = dfquar19rpvd['DESCR_RELACIONAMENTO'].map(str.strip)

<ipython-input-54-e94ab5bfc7a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfquar19rpvd['DESCR_RELACIONAMENTO'] = dfquar19rpvd['DESCR_RELACIONAMENTO'].map(str.strip)


In [55]:
dfquar19rpvd['DESCR_RELACIONAMENTO'].unique()

array(['CASAMENTO', 'CONHECIDO', 'UNIAO ESTAVEL', 'ENVOLVIMENTO AMOROSO',
       'PARENTESCO', 'VIZINHANCA', 'NENHUMA RELACAO', 'IGNORADO'],
      dtype=object)

In [56]:
dfquar19rpvd = dfquar19rpvd.set_index("DESCR_RELACIONAMENTO")

In [57]:
dfquar19rpvd = dfquar19rpvd.drop("CONHECIDO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("PARENTESCO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("VIZINHANCA", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("NENHUMA RELACAO", axis=0)
dfquar19rpvd = dfquar19rpvd.drop("IGNORADO", axis=0)

In [58]:
dfquar19rpvd.sample(5)

,CIDADE,DATA_COMUNICACAO_BO,FLAG_STATUS,VIOLENCIA_DOMESTICA,RUBRICA,DESCR_SUBTIPOLOCAL,IDADE_PESSOA,DESCR_PROFISSAO,DESCR_GRAU_INSTRUCAO
DESCR_RELACIONAMENTO,,,,,,,,,
UNIAO ESTAVEL,RIBEIRAO PRETO,2019-05-06,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),APARTAMENTO,28,VENDEDOR(A),2 Grau completo
UNIAO ESTAVEL,RIBEIRAO PRETO,2019-04-19,Consumado,VIOLÊNCIA DOMESTICA,Lesão corporal (art. 129),CASA,22,DESEMPREGADO,2 Grau completo
UNIAO ESTAVEL,RIBEIRAO PRETO,2019-05-08,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,49,FAXINEIRO(A),NaN
ENVOLVIMENTO AMOROSO,RIBEIRAO PRETO,2019-05-07,Consumado,VIOLÊNCIA DOMESTICA,Ameaça (art. 147),CASA,24,PRENDAS DOMESTICAS,2 Grau completo
CASAMENTO,RIBEIRAO PRETO,2019-05-31,Consumado,VIOLÊNCIA DOMESTICA,Injúria (art. 140),CASA,30,ADMINISTRADOR DE EMPRESAS,Superior completo


In [59]:
dfquar19rpvd.shape

(896, 9)

Com estes comandos, eu transformei a coluna "DESCR_RELACIONAMENTO" em índice para facilitar a filtragem por linhas, selecionando apenas os registros onde a vítima tinha algum relacionamento amoroso com o agressor. Farei o mesmo com o dataframe "dfquar20rpvd"

In [60]:
dfquar20rpvd['DESCR_RELACIONAMENTO'].unique()

array(['CASAMENTO                               ',
       'UNIAO ESTAVEL                           ',
       'ENVOLVIMENTO AMOROSO                    ',
       'PARENTESCO                              ',
       'CONHECIDO                               ',
       'NENHUMA RELACAO                         ', nan,
       'IGNORADO                                '], dtype=object)

In [61]:
dfquar20rpvd = dfquar20rpvd.dropna(subset=['DESCR_RELACIONAMENTO'])

In [62]:
dfquar20rpvd['DESCR_RELACIONAMENTO'] = dfquar20rpvd['DESCR_RELACIONAMENTO'].map(str.strip)

Por conter um valor NaN na coluna "DESCR_RELACIONAMENTO", eu não consegueria remover os espaços. Então eu excluí as linhas com esse valor e depois os removi.

In [63]:
dfquar20rpvd = dfquar20rpvd.set_index("DESCR_RELACIONAMENTO")

In [64]:
dfquar20rpvd = dfquar20rpvd.drop("CONHECIDO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("PARENTESCO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("NENHUMA RELACAO", axis=0)
dfquar20rpvd = dfquar20rpvd.drop("IGNORADO", axis=0)

In [65]:
dfquar20rpvd.shape

(569, 9)

In [66]:
dfquar19rpvd = dfquar19rpvd.reset_index()
dfquar20rpvd = dfquar20rpvd.reset_index()

Voltei para o índice padrão nos dois dataframes.

In [67]:
dfquar20rpvd['IDADE_PESSOA'].value_counts()

36    30
40    28
32    26
35    24
22    24
34    24
31    22
38    22
29    21
23    20
24    20
21    16
33    16
37    15
41    15
28    14
25    14
47    13
44    13
27    13
49    13
26    11
18    11
43    11
42    11
39    11
45    11
30    11
51     8
53     8
48     7
50     7
46     6
59     5
58     4
57     4
64     4
17     4
55     4
19     3
0      3
61     3
62     3
54     2
60     2
63     2
68     2
16     2
13     2
70     1
52     1
20     1
80     1
Name: IDADE_PESSOA, dtype: int64

Qual a idade mais frequente na coluna "IDADE_PESSOA".

In [68]:
dfquar20rpvd['RUBRICA'].value_counts()

Ameaça (art. 147)                                                                   271
Injúria (art. 140)                                                                  140
Lesão corporal (art. 129)                                                           111
Lesão corporal (art 129 § 9º)                                                        21
Dano (art. 163)                                                                      11
Difamação (art. 139)                                                                  6
Calúnia (art. 138)                                                                    4
Art.216B -Produzir, fotografar, filmar ou registrar, por qualquer meio, conteúdo      1
Dano qualificado (art. 163, par.único, I)                                             1
Homicídio qualificado (art. 121, §2o.)                                                1
A.I.-Lesão corporal (art. 129)                                                        1
Art. 213 - Estupro              

O mesmo para "RUBRICA".

In [69]:
dfquar19rp['RUBRICA'].value_counts()

Ameaça (art. 147)                                                                   945
Injúria (art. 140)                                                                  591
Lesão corporal (art. 129)                                                           326
Difamação (art. 139)                                                                107
Dano (art. 163)                                                                      77
Calúnia (art. 138)                                                                   53
Lesão corporal (art 129 § 9º)                                                        48
Art. 215A - Importunação Sexual                                                      27
Estupro de vulneravel (art.217-A)                                                    26
Maus-tratos (art. 136)                                                               21
Art. 213 - Estupro                                                                   18
A.I.-Ameaça (art. 147)          

O mesmo no dataframe maior, para comparação.

In [74]:
dfquar20['DESCR_GRAU_INSTRUCAO'].value_counts()

2 Grau completo                             1573
1 Grau incompleto                           1028
1 Grau completo                             1016
Superior completo                            815
2 Grau incompleto                            443
Superior incompleto                          160
Analfabeto                                    53
Name: DESCR_GRAU_INSTRUCAO, dtype: int64